In [163]:
import numpy as np
import scipy as sp

In [197]:
def rodrigez(fi,p):
    e=np.eye(3)
    px=np.array([[0,-p[2],p[1]],
                 [p[2],0,-p[0]],
                 [-p[1],p[0],0]])
    ppT=np.array([[p[0]**2,p[0]*p[1],p[0]*p[2]]
                 ,[p[1]*p[0],p[1]**2,p[1]*p[2]]
                 ,[p[2]*p[0],p[2]*p[1],p[2]**2]])
    return (1-np.cos(fi))*ppT+np.cos(fi)*e+np.sin(fi)*px

In [214]:
def A2AxisAngle(A):
# 1. Odredi jedinicni sopstveni vektor p za lambda=1
# 2. Odredi proizvoljan jedinicni vektor u normalan na p
# 3. Odredi u_prim koji je jednak A*u (u_prim je jedinicni)
# 4. Odredi fi koji je jednak arccos(u,u_prim)
# 5. Ako je mesoviti proizvod u,u_prim,p < 0, onda p=-p
# 1.
    sopstvene_vrednosti,sopstveni_vektori=sp.linalg.eig(A,left=True,right=False)
    p=sopstveni_vektori[:,2]
    p=np.array(p)
    p=np.real(p)
# 2. Gram Šmit ortogonalizacija
    u = np.array([1.0,2.0,3.0])
    u -= u.dot(p) * p / np.linalg.norm(p)**2 
    u /= np.linalg.norm(u)
# 3.
    u_prim=A@u
    u_prim/=np.linalg.norm(u_prim)
# 4.    
    fi=np.arccos(u.dot(u_prim)/(np.linalg.norm(u)*np.linalg.norm(u_prim)))
# 5.
    if np.dot(u,np.cross(u_prim,p)) < 0:
        p=-p
    return p,fi

In [215]:
A=np.array([[1,-8,-4],[4,4,-7],[8,-1,4]],dtype='float32')
A=A/9
print(A)
p,fi=A2AxisAngle(A)
rodrigez(fi,p)

[[ 0.11111111 -0.8888889  -0.44444445]
 [ 0.44444445  0.44444445 -0.7777778 ]
 [ 0.8888889  -0.11111111  0.44444445]]


array([[ 0.11111117, -0.88888896, -0.44444433],
       [ 0.44444442,  0.44444439, -0.77777785],
       [ 0.88888893, -0.11111099,  0.44444447]])

In [216]:
def A2Euler(A):
    if A[2][0] < 1:
        if A[2][0] > -1: #jedinstveno resenje
            psi=np.arctan2(A[1][0],A[0][0])
            teta=np.arcsin(-A[2][0])
            fi=np.arctan2(A[2][1],A[2][2])
            return psi,teta,fi
        else: # Ox3=-Oz
            psi=np.arctan2(-A[0][1],A[1][1])
            teta=np.pi/2
            fi=0
            return psi,teta,fi
    else: #Ox3=Oz
        psi=np.arctan2(-A[0][1],A[1][1])
        teta=-np.pi/2
        fi=0
        return psi,teta,fi

In [217]:
A2Euler(A)

(1.3258177, -1.0949141, -0.24497867)